In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from collections import Counter

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

n_children = 1000000 # n children to give
n_gift_type = 1000 # n types of gifts available
n_gift_quantity = 1000 # each type of gifts are limited to this quantity
n_gift_pref = 100 # number of gifts a child ranks
n_child_pref = 1000 # number of children a gift ranks
twins = math.ceil(0.04 * n_children / 2.) * 2    # 4% of all population, rounded to the closest number
triplets = math.ceil(0.005 * n_children / 3.) * 3    # 0.5% of all population, rounded to the closest number
ratio_gift_happiness = 2
ratio_child_happiness = 2



gift_pref = pd.read_csv('../input/child_wishlist_v2.csv',header=None).drop(0, 1).values
child_pref = pd.read_csv('../input/gift_goodkids_v2.csv',header=None).drop(0, 1).values




In [2]:
gift_pref.shape, child_pref.shape

((1000000, 100), (1000, 1000))

In [3]:
class Child(object):
    
    def __init__(self, idx, prefer):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for i in range(prefer.shape[0]):
            self.prefer_dict[prefer[i]] = [12*(prefer.shape[0] - i), -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 6*score
        else:
            self.prefer_dict[giftid] = [-6, 6*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
class Child_twin(object):
    
    def __init__(self, idx, prefer1, prefer2):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2))):
            score = 0
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = [3*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 3*score
        else:
            self.prefer_dict[giftid] = [-6, 3*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
class Child_triplet(object):
    
    def __init__(self, idx, prefer1, prefer2, prefer3):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2) + list(prefer3))):
            score = 0
            if p in list(prefer1):
                score += 2*(100 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(100 - list(prefer2).index(p))
            else:
                score -= 1
            if p in list(prefer3):
                score += 2*(100 - list(prefer3).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = [2*score, -6]
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid][1] = 2*score
        else:
            self.prefer_dict[giftid] = [-6, 2*score]
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, [-6, -6])
    
    
Children = []
for i in range(0, 5001, 3):
    Children.append(Child_triplet(i, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+1, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
    Children.append(Child_triplet(i+2, gift_pref[i], gift_pref[i+1], gift_pref[i+2]))
for i in range(5001, 45001, 2):
    Children.append(Child_twin(i, gift_pref[i], gift_pref[i+1]))
    Children.append(Child_twin(i+1, gift_pref[i], gift_pref[i+1]))
Children = Children + [Child(i, gift_pref[i]) for i in range(45001, 1000000)]

for j in range(1000):
    cf = child_pref[j]
    done_list = []
    for i in range(cf.shape[0]):
        if cf[i] <= 5000 and cf[i] not in done_list:
            if cf[i] % 3 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                cid3 = cf[i] + 2
                done_list.append(cid2)
                done_list.append(cid3)
            elif cf[i] % 3 == 1:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                cid3 = cf[i] + 1
                done_list.append(cid1)
                done_list.append(cid3)
            else:
                cid1 = cf[i] - 2
                cid2 = cf[i] - 1
                cid3 = cf[i]
                done_list.append(cid1)
                done_list.append(cid2)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            if cid3 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid3))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
            Children[cid3].add_gifts_prefer(j, score_)
        elif cf[i] <= 45000 and cf[i] not in done_list:
            if cf[i] % 2 == 1:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                done_list.append(cid2)
            else:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                done_list.append(cid1)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
        elif cf[i] > 45000:
            Children[cf[i]].add_gifts_prefer(j, 2*(cf.shape[0] - i))

In [4]:
from ortools.graph import pywrapgraph

W_CHILD = 1
W_GIFTS = 0

In [5]:
assignment = [-1]*1000000

In [6]:
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []


# triplets
for i in range(0, 5001, 3):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(3)
        unit_costs.append(1-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
            
# twins
for i in range(5001, 45001, 2):
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(2)
        unit_costs.append(1-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))
        
# other children
for i in range(45001, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        unit_costs.append(-W_CHILD*(Children[i].prefer_dict[g][0] + 6)-W_GIFTS*(Children[i].prefer_dict[g][1] + 6))

# add Arc
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
    
# children -> 1001000 : collection
for i in range(0, 5001, 3):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(3), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(2), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )
    
# gift -> 1001001 : dust_gift
for i in range(1000):
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    
# 1001001 -> 1001000 : dust_path
min_cost_flow.AddArcWithCapacityAndUnitCost(
    int(1001001), int(1001000), int(1000000), int(0)
)

101550693

In [7]:
# add Supply
for i in range(1000):
    min_cost_flow.SetNodeSupply(int(1000000+i), int(1000))

# children
for i in range(0, 5001, 3):
    min_cost_flow.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow.SetNodeSupply(int(i), int(0))

min_cost_flow.SetNodeSupply(int(1001001), int(0)) 
min_cost_flow.SetNodeSupply(int(1001000), int(-1000000)) 

In [8]:
min_cost_flow.Solve()

1

In [9]:
assignment = [-1]*1000000
twins_differ = []
triplets_differ = []

#for i in well_assigned:
#    assignment[i] = assignment_0[i]

for i in range(min_cost_flow.NumArcs()):
    if min_cost_flow.Flow(i) != 0 and min_cost_flow.Head(i) < 1000000:
        c = min_cost_flow.Head(i)
        g = min_cost_flow.Tail(i)
        f = min_cost_flow.Flow(i)

        if c >= 45001:
            assignment[c] = g - 1000000

        elif c >= 5001:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    twins_differ.append([c, c+1])
                else:
                    assignment[c+1] = g - 1000000
            elif f == 2:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
        else:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                elif assignment[c+1] == -1:
                    assignment[c+1] = g - 1000000
                else:
                    assignment[c+2] = g - 1000000
            elif f == 2:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    assignment[c+1] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                else:
                    assignment[c+1] = g - 1000000
                    assignment[c+2] = g - 1000000
            elif f == 3:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
                assignment[c+2] = g - 1000000
                
CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))

1 0 11739596260 44598 1617929090195333159939167951192
0.936301556826003
